In [25]:
import cv2
import numpy as np
import pandas as pd

def read_image(image_path):
    return cv2.imread(image_path)

def is_same_color(grid):
    return np.all(grid == grid[0, 0])

def is_white(grid):
    return np.all(grid == 247)

def process_grid(image, x, y):
    grid = image[y:y+15, x:x+15]
    is_white_grid = is_white(grid)
    is_same_color_grid = is_same_color(grid)
#     print(f"x: {x}, y: {y}, Grid: {grid}, is_white: {is_white_grid}, is_same_color: {is_same_color_grid}")
    if not is_white_grid and is_same_color_grid:
        return True, grid[0, 0]
    return False, None

def draw_square(image, x, y, text, color):
    cv2.rectangle(image, (x, y), (x+16, y+16), color, 1)
    # cv2.putText(image, str(text), (x+1, y+6), cv2.FONT_HERSHEY_SIMPLEX, 0.1, color, 1, cv2.LINE_AA)
    
def create_csv(dictionary, csv_name):
    df = pd.DataFrame(list(dictionary.items()), columns=["ID", "Color"])
    df.to_csv(csv_name, index=False)

def process_image(image_path):
    image = read_image(image_path)
    
    height, width, channels = image.shape
    puzzle = np.ones((height, width, channels), dtype=np.uint8) * 255
    key_dict = {}
    counter = 1

    for y in range(0, height-16, 1):
        for x in range(0, width-16, 1):
            valid, color = process_grid(image, x, y)    
            if valid:  
                pass
                # print(f"x: {x}, y: {y}, valid: {valid}, color: {type(color[0])}")      
            if valid:
                draw_square(puzzle, x, y, counter, (int(color[0]), int(color[1]), int(color[2])))
                key_dict[counter] = f"#{''.join([format(val, '02X') for val in color])}"
                counter += 1

    cv2.imwrite("/Users/senthilgandhi/Downloads/puzzle.png", puzzle)
    create_csv(key_dict, "/Users/senthilgandhi/Downloads/key.csv")

if __name__ == "__main__":
    image_path = "/Users/senthilgandhi/Downloads/output_image.png"
    process_image(image_path)


In [2]:
# print current working directory
import os
print(os.getcwd())

/


In [26]:
import cv2
import numpy as np
import pandas as pd
from typing import List, Tuple


def same_color(square: np.ndarray) -> bool:
    """Check if all pixels in the square have the same color."""
    return np.all(square == square[0, 0])


def get_square(image: np.ndarray, x: int, y: int, size: int) -> np.ndarray:
    """Extract a square of the given size from the image at position (x, y)."""
    return image[y:y + size, x:x + size]


def max_square_size(image: np.ndarray, x: int, y: int) -> int:
    """Find the maximum square size for a given position (x, y) in the image."""
    size = 1
    while x + size < image.shape[1] and y + size < image.shape[0]:
        square = get_square(image, x, y, size)
        if not same_color(square):
            break
        size += 1
    return size - 1


def draw_square(image: np.ndarray, x: int, y: int, size: int, color: Tuple[int, int, int]) -> None:
    """Draw a square with a specified size and color on the image."""
    image[y:y + size, x:x + size] = color


def find_squares(image: np.ndarray) -> List[Tuple[int, int, int]]:
    """Find the biggest non-overlapping squares of the same color inside the image."""
    squares = []
    for y in range(image.shape[0]):
        for x in range(image.shape[1]):
            size = max_square_size(image, x, y)
            if size > 0:
                squares.append((x, y, size))
                draw_square(image, x, y, size, (0, 0, 0))  # Black color to mark used pixels
    return squares


def draw_biggest_squares(image: np.ndarray) -> None:
    """Draw the biggest non-overlapping squares of the same color inside the image."""
    squares = find_squares(image)
    for x, y, size in squares:
        color = tuple(get_square(image, x, y, size)[0, 0])
        draw_square(image, x, y, size, color)


# Load the image using OpenCV
image_path = "/Users/senthilgandhi/Downloads/output_image.png"
image = cv2.imread(image_path)

# Call the main function to draw the biggest squares
draw_biggest_squares(image)

# Save the resulting image
output_path = "/Users/senthilgandhi/Downloads/puzzle.png"
cv2.imwrite(output_path, image)
